# SAM

Segment Anything és un projecte de **Meta** amb dues grans aportacions:

- Un gran conjunt de dades per a la segmentació d'imatges
- El model Segment Anything (SAM) com a model de base per a la segmentació d'imatges

Va ser introduït a l'article **Segment Anything** per Alexander Kirillov _et al._** (Abril 2023) [enllaç](https://arxiv.org/pdf/2304.02643.pdf)

Aquest model s'inspira en el camp de la NLP (_Natural Language Processing_), on la creació de models base (**
_foundation models_**) i els grans conjunts de dades (per valor de milers de milions de dades) s'han convertit en la manera habitual de fer feina.


Com ja sabem, la segmentació d'imatges té diversos usos, aquests inclouen: l'anàlisi d'imatges biomèdiques, l'edició de fotografies i la conducció autònoma, entre d'altres. Per resoldre qualsevol d'aquests problemes, cal entrenar models especialitzats per cada una de les tasques que hem citat (és més, per cada un dels subproblemes que es deriven de cada tasca i per cada cas en particular). Això requereix un ampli coneixement del domini del problema i el temps necessari per a la recollida de dades específiques, per no parlar de les hores d'entrenament i ajustamen que són necessàries per als models d'aprenentatge profund.

Enllaços:
- [Repositòri Oficial](https://github.com/facebookresearch/segment-anything)
- [Demo for dummies](https://segment-anything.com/demo)
- [Ultralytics1](https://docs.ultralytics.com/models/sam/) i [Ultralytics2](https://docs.ultralytics.com/models/sam2/)


In [ ]:
SAM.ipynb

## Model

SAM és un model d'aprenentatge profund (basat en _transformers_). I com amb qualsevol aprenentatge profund, s'ha entrenat en un gran nombre d'imatges i màscares: **més de mil milions de màscares en 11 milions d'imatges**.

Com SAM és una model base, i està preparat per segmentar qualsevol tipus d'imatge, pot rebre indicacions dels usuaris sobre quina àrea segmentar. Actualment podem proporcionar tres indicacions diferents a SAM:

- Indicar punts que contenen i no contenen el que volem segmentar.
- Dibuixant un quadre delimitador (_bounding box_).
- Dibuixant una màscara genearl sobre un objecte.

En l'article és parla de preparar un _promt_ per poder especificar el que es vol segmentar emprant text.

L'arquitectura de la xarxa és la següent:

![SAM](https://learnopencv.com/wp-content/uploads/2023/04/segment-anything-model.png)


Les caracterìstiques més importants del model (secció 3 de l'article) són:

**Encoder**. Vision Transformer (ViT) basat en la idea d'un MAE (Masked AutoEncoder) El codificador d'imatge s'executa una vegada per imatge i es pot aplicar abans de sol·licitar el
_prompt_ al model.

**Codificador de _prompts_**. S'en consideren dos tipus: dispersos (punts, _bounding boxes_, text) i densos (màscares). Representen els punts i les _bounding boxes_ mitjançant codificacions posicionals amb altres informacions apreses per a cada tipus d'_input_ i text de forma lliure amb un codificador de text anomenat CLIP. Les indicacions denses (és a dir, les màscares) s'incorporen mitjançant l'ús convolucions i es sumen (concatenen) amb la codificació obtinguda de l'encoder de la imatge.

**Descodificador de màscares**. El descodificador de màscara mapeja de manera eficient els _embeddings_ d'una imatge i el resultat del codificador de _prompts_ utilitzant una modificació d'un bloc descodificador Transformer 

**Resolució de l'ambigüitat**. Amb una sortida, el model farà una mitjana de diverses màscares vàlides si se li dóna una indicació ambigua. Per solucionar-ho, modifiquem el model per predir múltiples màscares de sortida per a un sol _prompt_ d'entreada. Experimentalment s'ha arribat a la conclusió que 3 màscares de sortida de màscara són suficients per abordar els casos més habituals (les màscares imbricades solen tenir tres profunditats com a màxim: senceres, parcials i subparts). 

**Eficiència**. El disseny global del model està motivat en gran mesura per l'eficiència. Donat un _embedding_ precalculat d'una imatge, el codificador de _promts_ i el descodificador de màscares s'executen en un navegador web, a la CPU, en ∼50 ms. Aquest rendiment en temps d'execució permet una indicació interactiva en temps real del model.

**Pèrdues i entrenament.** Es supervisa la predicció de les màscares amb la combinació lineal d'una funció de pèrdua focal [enllaç](https://paperswithcode.com/method/focal-loss) i una funció de pèrdua de dice (ja emprada al model YOLO). L'entrenament es realitza utilitzant una barreja d'indicacions geomètriques, es simula un entrenament interactiu mitjançant un mostreig aleatori de prompts en 11 rondes per màscara.


## Dades

Com ja sabem, la base de qualsevol model d'aprenentatge profund innovador és el conjunt de dades en què s'ha entrenat. El conjunt de dades de SAM conté més d'**11 milions d'imatges i 1.100 milions de màscares**. El conjunt de dades final s'anomena conjunt de dades SA-1B.

Segurament es necessita aquest conjunt de dades per entrenar un model de capacitat de Segment Anything. Però també sabem que aquests conjunts de dades no existeixen i que és impossible anotar manualment tantes imatges.Per tant es necessitar l'ajut de SAM per anotar el conjunt de dades: Els anotadors de dades van utilitzar SAM per anotar imatges de manera interactiva i les dades anotades es van utilitzar per entrenar SAM. Aquest procés es va repetir, cosa que va donar lloc al motor de dades en bucle de SAM.

Aquest motor de dades + formació del SAM al conjunt de dades té tres etapes:

- Etapa Manual Assistida
- Etapa semiautomàtica
- Etapa totalment automàtica

En la primera etapa, els anotadors van utilitzar un model SAM prèviament entrenat per segmentar objectes de manera interactiva en imatges al navegador. Els _embeddings_ de les imatges es van calcular prèviament per fer que el procés d'anotació fos fluid i en temps real. Després de la primera etapa, el conjunt de dades constava de 4,3 milions de màscares a partir  de 120.000 imatges. El model Segment Anything es va tornar a entrenar en aquest conjunt de dades.

En la segona etapa semiautomàtica, els objectes destacats ja estaven segmentats mitjançant SAM. Els anotadors també van anotar objectes menys destacats que no tenien anotació. Aquesta etapa va donar lloc a 5,9 milions de màscares addicionals en 180.000 imatges en les quals es va tornar a entrenar SAM.

A l'etapa "totalment automàtica" final, l'anotació la va fer íntegrament SAM. En aquesta etapa, ja s'havia entrenat el model en més de 10 milions de màscares que ho van fer possible. La generació automàtica de màscares es va aplicar 11M a imatges , donant lloc a 1,1B  màscares.
En paraules del seus creadors:

"La versió final del conjunt de dades Segment Anything el converteix en el conjunt de dades de segmentació d'imatges més gran disponible públicament. En comparació amb OpenImages V5, hi ha 6 vegades més imatges i 400 vegades més màscares al conjunt de dades."

## SAM2

SAM2 presenta una evolució de SAM, va ser publicat l'octubre de l'any 2024. [SAM 2: Segment Anything in Images and Videos](https://arxiv.org/pdf/2408.00714)

SAM2 pot segmentar qualsevol objecte en qualsevol vídeo o imatge, fins i tot per a objectes i dominis visuals que no hagi vist anteriorment, permetent una àmplia gamma de casos d'ús sense adaptació personalitzada.

Meta distibueix:
- El codi i els pesos SAM 2, que s'estan de codi obert sota una llicència permissiva d'Apache 2.0.
- El conjunt de dades SA-V, que té 4,5 vegades més vídeos i 53 vegades més anotacions que el conjunt de dades de segmentació de vídeo més gran existent. - Aquesta versió inclou ~ 51.000 vídeos del món real amb més de 600.000 mascares. 
- Una [demo](https://sam2.metademolab.com/) que permet la segmentació interactiva en temps real de vídeos curts i aplica efectes de vídeo a les prediccions del model.



L'arquitectura de SAM2 s'ha adaptat per poder processar videos, però també s'ha treballat en reduïr la seva mida. En la [següent taula](https://docs.ultralytics.com/models/sam-2/#sam-2-comparison-vs-yolov8) en podem veure una comparativa molt interessant.

## Tutorial

Hi ha 3 models diferents preentrenats de SAM:

- ViT-B SAM
- ViT-L SAM
- ViT-H SAM

Per altra banda, tenim 4 models diferents de SAM2 (i de SAM2.1):

- sam2_hiera_tiny
- sam2_hiera_small
- sam2_hiera_base_plus
- sam2_hiera_large



Tal com passava amb YOLO es pot emprar SAM i SAM2 descarregant el model i/o els pesos desde els seus resperctius repositoris:

- SAM [enllaç](https://github.com/facebookresearch/segment-anything#model-checkpoints).

- SAM2 [enllaç](https://github.com/facebookresearch/sam2).

Aquests models estàn implementats amb Pytorch i tenir la possibilitat d'accedir al codi font ens podría servir per tasques d'investigació.

Per altra banda la empresa Ultralytics ha portat aquests dos models al seu entorn, amb la limitació que només podem fer predicció amb les condicions/restriccions de l'entron, però també aprofitant la seva simplesa.

### Feina a fer
0. Segmentar una única cel·lula a la imatge anomenada _cells.jpg_ que es troba a la carpeta _imgs_ mitjançant la selecció d'un punt i dibuixar la seva _bounding box_.
1. Contar el nombre de cel·lules a la imatge anomenada _cells.jpg_ que es troba a la carpeta _imgs_.

![celules](imgs/cells.jpg "celules")

2. Segmentar un dels tres joves que apareixen a la imatge _str.jpg_, mostrar el resultat de la segmentació posant de color negre tot el que no sigui part de la segmentació. L'exercici és més senzill si emprau una bounding box.

![str](imgs/str.jpg "str")

3. Segmentar els tres joves emprant YOLO i una de les dues versions de SAM.

A continuació teniu un exemple d'execució útil tant per SAM com per SAM2

In [ ]:
from ultralytics.models.sam import Predictor as SAMPredictor
from ultralytics.models.sam import SAM2VideoPredictor ## Per fer segmentació sobre vídeos

# Diccionari amb la configuració del predictor
overrides = dict(imgsz=1024, model="¿?") # Si teniu moltes limitacions hard: mobile_sam.pt
predictor = SAMPredictor(overrides=overrides)
predictor.reset_image()
# Assignam una imatge al predictor
predictor.set_image("¿?")  #pot ser un path o una url o un ndarray (numpy)


# Exemple 1: Segmentació indicant un punt d'interés amb coordenades (x,y)
#results = predictor(points=[x, y], labels=[1])  # label 1 indica objecte; label 0 indica fons

# Exemple 2: Segmentació indicant dos punts d'interés amb coordenades (x1,y1); (x2,y2)
#results = predictor(points=[[x1, y1], [x2, y2]], labels=[[1, 1]])

# Exemple 3
#results = predictor(points=[[[x1, y1], [x2, y2]]], labels=[[1, 0]])

# Exemple 4: Segmentació indicant caixa
#results = predictor(bboxes=[[[xi,yi], [xi, yf]]] )

# Exemple 4: Segmentació indicant caixa
#results = predictor(bboxes=[[[xi,yi], [xi, yf]]], points=[x,y], labels=[z] )
# Reset image
#predictor.reset_image()

També es pot fer una predicció de tota la imatge. No és recomanable ja que el temps d'inferéncia augmenta considerablement però pot ser d'utilitat en certs casos

In [ ]:
from ultralytics import SAM

# Carregam un model
model = SAM("mobile_sam.pt")

# Display model information (optional)
model.info()

# Executar l'inferència tal com ho feim a pytorch. Aquí podem passar la imatge directament.
model("imgs/cells.jpg")